# CRF postprocessing and evaluation for cancer detection method

In [ ]:
# post-processing
from skimage import io
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 
%matplotlib inline
from PIL import Image

from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

from keras.callbacks import EarlyStopping
import keras
from keras.models import *
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Dropout, concatenate, Reshape, Permute, Lambda, BatchNormalization
from keras.activations import softmax
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, History
from keras import backend as K
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
from keras_preprocessing import image

import glob 
import cv2
from scipy import misc
import sys
# import shutil

from zipfile import ZipFile 

from keras.layers import Lambda
from keras import backend as K

import imageio
import numpy as np
import random

import datetime
import json
from keras.models import model_from_json

import imgaug as ia
from imgaug import augmenters as iaa
import imageio

import pandas as pd
import six
import csv
 

ia.seed(1)

In [ ]:
# Mask generator into one hot

patch_size = 512
batch_size = 16

class MasksBatchGenerator(DirectoryIterator):
    
    def __init__(self,
                directory, 
                image_data_generator,
                target_size = (patch_size, patch_size),
                num_channels = 1,
                classes=None, 
                class_mode=None,
                batch_size=batch_size, shuffle=True,
                seed=None,
                data_format=None,
                save_to_dir=None,
                save_prefix='',
                save_format='png',
                follow_links=False,
                subset=None,
                interpolation='nearest',
                preprocessing_function= None):

        super(MasksBatchGenerator,self).__init__(directory, 
                image_data_generator,                                              
                target_size=target_size,                      
                classes=classes,
                class_mode=class_mode,
                batch_size=batch_size,
                shuffle=shuffle,
                seed=seed,
                data_format=data_format,
                save_to_dir=save_to_dir,
                save_prefix=save_prefix,
                save_format=save_format,
                follow_links=follow_links,
                subset=subset,
                interpolation=interpolation)

        self.image_shape = (target_size[0],target_size[1],num_channels)
        self.preprocessing_function = preprocessing_function
        
    def _get_batches_of_transformed_samples(self, index_array):
        
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype = K.floatx())
        
        for i, j in enumerate(index_array):
            
            fname = self.filenames[j]

            img = cv2.cvtColor(cv2.imread(os.path.join(self.directory, fname)), cv2.COLOR_BGR2RGB)
            
            if self.preprocessing_function:
                img = self.preprocessing_function(img)
                
            batch_x[i] = img
            
        return batch_x

In [ ]:
# Patch generator

class PatchBatchGenerator(DirectoryIterator):
    
    def __init__(self,
                directory, 
                image_data_generator,
                target_size = (patch_size, patch_size),
                num_channels = 3,
                color_mode = "rgb",
                classes=None, 
                class_mode=None,
                batch_size=batch_size, shuffle=True,
                seed=None,
                data_format=None,
                save_to_dir=None,
                save_prefix='',
                save_format='jpg',
                follow_links=False,
                subset=None,
                interpolation='nearest',
                preprocessing_function= None):

        super(PatchBatchGenerator,self).__init__(directory, 
                image_data_generator,
                target_size=target_size,                      
                classes=classes,
                class_mode=class_mode,
                batch_size=batch_size,
                shuffle=shuffle,
                seed=seed,
                data_format=data_format,
                save_to_dir=save_to_dir,
                save_prefix=save_prefix,
                save_format=save_format,
                follow_links=follow_links,
                subset=subset,
                interpolation=interpolation)

        self.image_shape = (target_size[0],target_size[1],num_channels)
        
    def _get_batches_of_transformed_samples(self, index_array):
        
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype = K.floatx())
        
        for i, j in enumerate(index_array):
            
            fname = self.filenames[j]
            img = cv2.cvtColor(cv2.imread(os.path.join(self.directory, fname)), cv2.COLOR_BGR2RGB)        

            batch_x[i] = img/255.0
            
        return batch_x

In [ ]:
# Mask generator into one hot with statial augmentation


class MasksBatchGeneratorAug(DirectoryIterator):
    
    def __init__(self,
                directory, 
                image_data_generator,
                target_size = (patch_size, patch_size),
                num_channels = 1,
                classes=None, 
                class_mode=None,
                batch_size=batch_size, shuffle=True,
                seed=None,
                data_format=None,
                save_to_dir=None,
                save_prefix='',
                save_format='png',
                follow_links=False,
                subset=None,
                interpolation='nearest',
                preprocessing_function= None,
                preprocessing_function_space= None):

        super(MasksBatchGeneratorAug,self).__init__(directory, 
                image_data_generator,
                target_size=target_size,                      
                classes=classes,
                class_mode=class_mode,
                batch_size=batch_size,
                shuffle=shuffle,
                seed=seed,
                data_format=data_format,
                save_to_dir=save_to_dir,
                save_prefix=save_prefix,
                save_format=save_format,
                follow_links=follow_links,
                subset=subset,
                interpolation=interpolation)

        self.image_shape = (target_size[0],target_size[1],num_channels)
        self.preprocessing_function_space = preprocessing_function_space
        self.preprocessing_function = preprocessing_function
        
    def _get_batches_of_transformed_samples(self, index_array):
        
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype = K.floatx())
        
        for i, j in enumerate(index_array):
            
            fname = self.filenames[j]
            img = cv2.cvtColor(cv2.imread(os.path.join(self.directory, fname)), cv2.COLOR_BGR2RGB)
            
            if self.preprocessing_function_space:
                img= self.preprocessing_function_space(img)
            
            if self.preprocessing_function:
                img = self.preprocessing_function(img)
                
            batch_x[i] = img
            
        return batch_x

In [ ]:
# Patch generator with statial augmentation


class PatchBatchGeneratorAug(DirectoryIterator):
    
    def __init__(self,
                directory, 
                image_data_generator,
                target_size = (patch_size, patch_size),
                num_channels = 3,
                color_mode = "rgb",
                classes=None, 
                class_mode=None,
                batch_size=batch_size, shuffle=True,
                seed=None,
                data_format=None,
                save_to_dir=None,
                save_prefix='',
                save_format='jpg',
                follow_links=False,
                subset=None,
                interpolation='nearest',
                preprocessing_function_space= None,
                preprocessing_function= None):

        super(PatchBatchGeneratorAug,self).__init__(directory, 
                image_data_generator,
                target_size=target_size,                      
                classes=classes,
                class_mode=class_mode,
                batch_size=batch_size,
                shuffle=shuffle,
                seed=seed,
                data_format=data_format,
                save_to_dir=save_to_dir,
                save_prefix=save_prefix,
                save_format=save_format,
                follow_links=follow_links,
                subset=subset,
                interpolation=interpolation)

        self.image_shape = (target_size[0],target_size[1],num_channels)
        self.preprocessing_function_space = preprocessing_function_space
        
    def _get_batches_of_transformed_samples(self, index_array):
        
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype = K.floatx())
        
        for i, j in enumerate(index_array):
            
            fname = self.filenames[j]
            img = cv2.cvtColor(cv2.imread(os.path.join(self.directory, fname)), cv2.COLOR_BGR2RGB)
            
            if self.preprocessing_function_space:
                img= self.preprocessing_function_space(img)
                
            batch_x[i] = img/255.0
            
        return batch_x

## Functions

In [ ]:
# Open image function

def open_image(path):
  newImage = Image.open(path)
  return newImage

In [ ]:
## 0 is background, 1 is healthy, 2 is cancer
def rgb_to_onehot_3cls(rgb_arr):
    
    color_dict = {0: [255, 255, 255], # white- background  bgr rgb GOOD
              1: [255, 0, 255], # magenta - normal 
              2: [102, 102, 102], # gray- other GOOD
              3: [50, 160, 50], # also something green- acinary tumour
              4: [0, 255, 0], # green lime- lepidic tumour GOOD
              5: [0, 0, 0], # black- solid tumour GOOD
              6: [0, 75, 0], # - darker gree tumour
              7: [130, 170, 130], # light green- micropapollary tumour 
                 }          
             
    num_classes = len(color_dict)-5
    shape = rgb_arr.shape[:2]
    arr = np.zeros( shape, dtype=np.int8 )
    
    for i, cls in color_dict.items():
        
        if i == 0:
            
            mask0 = rgb_arr[:, :, 0] == cls[0]
            mask1 = rgb_arr[:, :, 1] == cls[1]
            mask2 = rgb_arr[:, :, 2] == cls[2]
            mask = np.logical_and(np.logical_and(mask0, mask1), mask2)
            arr[mask] = i
            
        elif i == 1 or i == 2:
            
            mask0 = rgb_arr[:, :, 0] == cls[0]
            mask1 = rgb_arr[:, :, 1] == cls[1]
            mask2 = rgb_arr[:, :, 2] == cls[2]
            mask = np.logical_and(np.logical_and(mask0, mask1), mask2)
            arr[mask] = 1
            
        elif i == 3 or i == 4 or i == 5 or i == 6 or i == 7:
            
            mask0 = rgb_arr[:, :, 0] == cls[0]
            mask1 = rgb_arr[:, :, 1] == cls[1]
            mask2 = rgb_arr[:, :, 2] == cls[2]
            mask = np.logical_and(np.logical_and(mask0, mask1), mask2)
            arr[mask] = 2
        
    arr_vec=np.zeros((arr.shape[0], arr.shape[1], num_classes))
    
    for i in range(num_classes):
        
        mask = (arr == i)
        arr_vec[:, :, i][mask] = 1 
        
    return arr_vec

In [ ]:
def CRF_to_onehot(arr):
    color_dict = {0: (1,   0, 0),       
                  1: (0, 1, 0),   
                  2: (0, 0,   1)}    
    arr= np.squeeze(arr)
    single_layer = arr
    output = np.zeros((512,512,3))
    for k in color_dict.keys():
        output[single_layer==k] = color_dict[k]
    return np.uint8(output)

In [ ]:
def onehot_to_rgb_3cls(onehot):
    
    color_dict = {0: [255, 255, 255], # white- background  bgr rgb
              1: [255, 0, 255], # non-cancer -blue
              2: [0, 0, 0], # cancer -red
                 }
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros( onehot.shape[:2]+(3,) )
    for k in color_dict.keys():
        output[single_layer==k] = color_dict[k]
    return np.uint8(output)

In [ ]:
def brightness_augmentation(factor):
    factor = factor
    
    def augment(img): 

        hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) #convert to hsv
        hsv = np.array(hsv, dtype=np.float64)
        hsv[:, :, 2] = hsv[:, :, 2] * (factor + np.random.uniform()) #scale channel V uniformly
        hsv[:, :, 2][hsv[:, :, 2] > 255] = 255 #reset out of range values
        rgb = cv2.cvtColor(np.array(hsv, dtype=np.uint8), cv2.COLOR_HSV2RGB)
        
        augment = rgb
        return augment
    
    return augment 

In [ ]:
def weighted_categorical_crossentropy(weights):
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        
        return loss
    
    return loss

In [ ]:
# Measurements

def precision(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):

    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
        
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):

    return fbeta_score(y_true, y_pred, beta=1)


In [ ]:
# Metrics visualisation function

def render_table(data, col_width = 3.0, row_height = 0.625, font_size = 14,
                     header_color = '#F8F8FF', row_colors = ['w'], edge_color = '#D3D3D3',
                     bbox = [0, 0, 1, 1], header_columns = 0,
                     ax = None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText = data.values, bbox = bbox, colLabels = data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight = 'bold', color = '#000000')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
    return ax

## Model implementation

In [ ]:
#### Model parameters (inputs)

n_classes = 3
patch_size = 512 #both height and width 

batch_size = 16
n_epoch = 100

learning_rate = 0.01
patience = 100

# type_loss_function = keras.losses.categorical_crossentropy
class_weights = [1,1,2]
type_loss_function = weighted_categorical_crossentropy(class_weights)
# cells to be measured 
lim = 100

# Data directory
Test_path = "/home/guest/DL/Teodora/Project/Test516"
Train_path = "/home/guest/DL/Teodora/Project/Train516"
Val_path = "/home/guest/DL/Teodora/Project/Validation516"

In [ ]:
#U-net implementation

inputs = Input((patch_size, patch_size, 3))
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', dilation_rate = 2)(inputs)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', dilation_rate = 2)(conv1)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

print(conv1.shape)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)


print(conv2.shape)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

print(conv3.shape)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

print(conv4.shape)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
print(conv5.shape)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv10 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

conv11 = Conv2D(n_classes, (1, 1), activation='linear')(conv10)
print(conv10.shape)
out=conv11
out = Lambda(lambda x: softmax(x, axis = 3))(conv11)

model = Model(inputs=[inputs], outputs=[out])

# keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)#learning rate optimisation
model.compile(optimizer = 'adam', loss = type_loss_function, metrics=['accuracy', recall, precision, fmeasure])

# model.summary()

## Test Data

In [ ]:
patch_arg = dict()
mask_arg = dict()
# PatchBatchGenerator= ImageDataGenerator()

test_patch_generator = PatchBatchGenerator(
                                            directory= Test_path+'/Patches',
                                            image_data_generator=ImageDataGenerator,
                                            target_size=(patch_size, patch_size),
                                            color_mode="rgb",
                                            batch_size=1,
                                            class_mode=None,
                                            shuffle=False,
                                            seed=1)
    
    
test_mask_generator = MasksBatchGenerator(
                                            directory= Test_path+'/Masks',
                                            image_data_generator=ImageDataGenerator,
                                            target_size=(patch_size, patch_size),
                                            num_channels=n_classes,
                                            class_mode=None,
                                            shuffle=False,
                                            batch_size=1,
                                            seed=1,
                                            preprocessing_function =  rgb_to_onehot_3cls)   

print('Test sets ready')

test_only_patch_generator = test_patch_generator
test_generator = zip(test_patch_generator, test_mask_generator)

In [ ]:
n_test_samples  = len([name for name in os.listdir(Test_path + '/Patches/Group1/')])
print (n_test_samples)

## Test evaluation

In [ ]:
#Load model
exp='A'

weights_path = os.path.dirname(os.path.realpath("Train")) + '/Weights_data_cross'

weights_file = weights_path + '/' + 'Weights.h5'

#load weights
model.load_weights(weights_file)


print("Weights are loaded into model")

In [ ]:
test_scores = model.evaluate_generator(test_generator, n_test_samples, max_queue_size = 1)

#Loss, accuracy, recall, precission, fmeasure
print(test_scores) 

test_res = model.predict_generator(test_only_patch_generator, n_test_samples) # test set prediction map

In [ ]:
import sys

import pydensecrf.densecrf as dcrf

from pydensecrf.utils import compute_unary, create_pairwise_bilateral, \
    create_pairwise_gaussian, softmax_to_unary, unary_from_softmax

import skimage.io as io
def load_patch(patch):
    im = Image.open(patch )
    im.load()
    data = np.asarray( im, dtype="uint32" )
    return data

n=878
masks = [cv2.imread(file) for file in sorted(glob.glob(os.path.join(Test_path+'/Masks/Group1/', '*-labels.png')))]
patches = [file for file in sorted(glob.glob(os.path.join(Test_path+'/Patches/Group1/', '*jpg')))]
CRF_eval=[]
CRF_res=[]

# for i in range(0,n_test_samples):
for i in range(n,n+1):  
    image = load_patch(patches[i])
    image = image/.255
#     print(image.shape)

    final_probabilities = test_res[i,:,:,:]
#     print(len(final_probabilities[1,:,:])*len(final_probabilities[:,1,:]))
    
    processed_probabilities = final_probabilities.squeeze()

    softmax = processed_probabilities.transpose((2, 0, 1))

    # The input should be the negative of the logarithm of probability values
    # Look up the definition of the softmax_to_unary for more information
    unary = unary_from_softmax(softmax)

    # The inputs should be C-continious -- we are using Cython wrapper
    unary = np.ascontiguousarray(unary)

    d = dcrf.DenseCRF2D(image.shape[0], image.shape[1], 3)
    
    d.setUnaryEnergy(unary)
    d.addPairwiseGaussian(sxy=(5,5), compat=50, kernel=dcrf.FULL_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)

    # This adds the color-dependent term, i.e. features are (x,y,r,g,b).
    d.addPairwiseBilateral(sxy=(5,5), srgb=(5,5,5), rgbim=cv2.cvtColor(cv2.imread(patches[i]), cv2.COLOR_BGR2RGB), compat=100, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)

    Q = d.inference(5)

    res1 = np.argmax(Q, axis=0).reshape((image.shape[0], image.shape[1]))

    CRF_eval.append(res1)
    
    #model segmentation
    test_res_sample =  np.array(test_res[i,:,:,:])
    most_prob = np.zeros(shape = [patch_size, patch_size, n_classes])

    most_prob[:,:,0] = (np.array(test_res_sample[:,:,0] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
    most_prob[:,:,1] = (np.array(test_res_sample[:,:,1] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
    most_prob[:,:,2] = (np.array(test_res_sample[:,:,2] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
    
    # Prediction annotations
    
    predicted_segmentation = onehot_to_rgb_3cls(most_prob)
    pred1 = predicted_segmentation
    
    #CRF segmentation
    res =np.array(CRF_to_onehot(res1))
    
    most_prob2 = np.zeros(shape = [patch_size, patch_size, n_classes])
    most_prob3 = np.zeros(shape = [patch_size, patch_size, n_classes])
    most_prob4 = np.zeros(shape = [patch_size, patch_size, n_classes])

    most_prob2[:,:,0] = (np.array(res[:,:,0] == np.fmax(np.fmax(res[:,:,0], res[:,:,1]),((res[:,:,2]))),dtype = 'float32'))
    most_prob2[:,:,1] = (np.array(res[:,:,1] == np.fmax(np.fmax(res[:,:,0], res[:,:,1]),((res[:,:,2]))),dtype = 'float32'))
    most_prob2[:,:,2] = (np.array(res[:,:,2] == np.fmax(np.fmax(res[:,:,0], res[:,:,1]),((res[:,:,2]))),dtype = 'float32'))

    most_prob3[:,:,0] = most_prob2[:,:,0]
    most_prob3[:,:,1] = cv2.morphologyEx(most_prob2[:,:,1], cv2.MORPH_CLOSE, np.ones((5,5),np.uint8),iterations = 2)
    most_prob3[:,:,2] = cv2.morphologyEx(most_prob2[:,:,2], cv2.MORPH_CLOSE, np.ones((5,5),np.uint8),iterations = 2)

    
    predicted_segmentation2 = onehot_to_rgb_3cls(most_prob2)
    pred2 = predicted_segmentation2
    
    predicted_segmentation3 = onehot_to_rgb_3cls(most_prob3)
    pred3 = predicted_segmentation3
    
    CRF_res.append(pred2)
    
#Extract contours
    
    magenta = np.array([255, 0, 255])
    black = np.array([0, 0, 0])
    white = np.array([255, 255, 255])
    green = np.array([0, 255, 0])

    only_magenta = cv2.inRange(pred2, magenta, magenta)
    only_black = cv2.inRange(pred2, black, black)
#     only_white = cv2.inRange(pred2, white, white)

    c_m, _ = cv2.findContours(only_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c_b, _  = cv2.findContours(only_black, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     c_w, _  = cv2.findContours(only_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    im_crf = np.copy(cv2.cvtColor(cv2.imread(patches[n]), cv2.COLOR_BGR2RGB))

    cv2.drawContours(im_crf, c_m, -1, (255, 0, 255), 5)
    cv2.drawContours(im_crf, c_b, -1, (0, 0, 0), 3)
#   cv2.drawContours(im, c_g, -1, (0, 255, 0), 3)
    
    magenta = np.array([255, 0, 255])
    black = np.array([0, 0, 0])
    white = np.array([255, 255, 255])
    green = np.array([0, 255, 0])

    only_magenta = cv2.inRange(pred1, magenta, magenta)
    only_black = cv2.inRange(pred1, black, black)
    only_white = cv2.inRange(pred1, white, white)

    c_m, _ = cv2.findContours(only_magenta, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c_b, _  = cv2.findContours(only_black, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     c_w, _  = cv2.findContours(only_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    im = np.copy(cv2.cvtColor(cv2.imread(patches[n]), cv2.COLOR_BGR2RGB))

    cv2.drawContours(im, c_m, -1, (255, 0, 255), 5)
    cv2.drawContours(im, c_b, -1, (0, 0, 0), 3)
    
    
 # Ground truth annotations
    mask_sample = cv2.cvtColor(masks[n], cv2.COLOR_BGR2RGB)

    gray = np.array([102, 102, 102])
    green_acinar = np.array([50, 160, 50])
    green_pap = np.array([0, 75, 0])
    green_mpap = np.array([130, 170, 130])
    
    only_magenta_mask = cv2.inRange(mask_sample, magenta, magenta)
    only_black_mask = cv2.inRange(mask_sample, black, black)
    only_green_mask = cv2.inRange(mask_sample, green, green)
    only_gray_mask = cv2.inRange(mask_sample, gray, gray)
    only_green_ac_mask = cv2.inRange(mask_sample, green_acinar, green_acinar)
    only_green_pa_mask = cv2.inRange(mask_sample, green_pap, green_pap)
    only_green_mpa_mask = cv2.inRange(mask_sample, green_mpap, green_mpap)
#     only_white = cv2.inRange(mask_sample, white, white)

    c_m, _ = cv2.findContours(only_magenta_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c_b, _  = cv2.findContours(only_black_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c_green, _  = cv2.findContours(only_green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c_gray, _  = cv2.findContours(only_gray_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c_green_pa, _  = cv2.findContours(only_green_pa_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c_green_mpa, _  = cv2.findContours(only_green_mpa_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c_green_ac, _  = cv2.findContours(only_green_ac_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     c_w, _  = cv2.findContours(only_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    im_mask = np.copy(cv2.cvtColor(cv2.imread(patches[n]), cv2.COLOR_BGR2RGB))

    cv2.drawContours(im_mask, c_m, -1, (255, 0, 255), 5)
    cv2.drawContours(im_mask, c_gray, -1, (255, 0, 255), 5) 
    cv2.drawContours(im_mask, c_b, -1, (0, 0, 0), 3)
    cv2.drawContours(im_mask, c_green, -1, (0, 0, 0), 3) 
    cv2.drawContours(im_mask, c_green_pa, -1, (0, 0, 0), 3)
    cv2.drawContours(im_mask, c_green_mpa, -1, (0, 0, 0), 3)
    cv2.drawContours(im_mask, c_green_ac, -1, (0, 0, 0), 3)
   # cv2.drawContours(im, c_w, -1, (122, 122, 122), 1)
    #cv2.imwrite("contours_blue.png", im)
    
    (f, [ax00, ax0, ax1, ax2, ax4]) = plt.subplots(1, 5, figsize = (25, 5))
    ax00.imshow(cv2.cvtColor(cv2.imread(patches[i]), cv2.COLOR_BGR2RGB))
    ax0.imshow(pred1)
    ax0.set_title('Segmentation with Unet') 
    ax1.imshow(pred2)
    ax1.set_title('Segmentation with CRF post-processing')
    ax2.imshow(pred3)
    ax2.set_title('Segmentation with CRF post-processing and closing')
#         ax3.imshow(pred4)
#         ax3.set_title('Segmentation with CRF post-processing, closing and dilation')
    ax4.imshow(cv2.cvtColor(masks[i], cv2.COLOR_BGR2RGB))
    ax4.set_title('Ground-Truth Annotation')
    plt.show()
    plt.clf()

    cv2.imwrite('figure.png',cv2.cvtColor(cv2.imread(patches[n]), cv2.COLOR_BGR2RGB))
    cv2.imwrite('fig-pred-annot.png',im)
    cv2.imwrite('fig-CRF-annot.png',im_crf)
    cv2.imwrite('fig-gt-annot.png',im_mask)
    cv2.imwrite('map-CRF-annot.png',pred2)
    cv2.imwrite('map-pred-annot.png',pred1)
    cv2.imwrite('map-gt-annot.png',cv2.cvtColor(masks[i], cv2.COLOR_BGR2RGB))
    
    

In [ ]:
plt.clf()

In [ ]:
# CRF_res= np.array(CRF_res)

# print(CRF_res[1])
# CRF_res.shape
# CRF_res[2,:,:,2]

## Metrics per group of patchs

In [ ]:
import math
# Calculate precision and recall per tissue type
epsi=0.00000001
masks = [cv2.imread(file) for file in sorted(glob.glob(os.path.join(Test_path+'/Masks/Group1/', '*-labels.png')))]

CRF_res= np.array(CRF_res)

for i in range(0, len(masks)):
    #b,g,r = cv2.split(masks[i])
    masks[i] = (cv2.cvtColor(masks[i], cv2.COLOR_BGR2RGB))

lim=n_test_samples

precision_background_list=[]
precision_cancer_list=[]
precision_normal_list=[]

recall_background_list=[]
recall_cancer_list=[]
recall_normal_list=[]

f1_background_list=[]
f1_cancer_list=[]
f1_normal_list=[]
list_3cls=[]

for n in range(0, lim): 
    ground_truth = rgb_to_onehot_3cls(masks[n])
    if(np.sum(np.sum(ground_truth[:,:,0]))!=0 and np.sum(np.sum(ground_truth[:,:,1]))!=0 and np.sum(np.sum(ground_truth[:,:,2]))!=0):
        list_3cls.append(n)

list_3cls = np.array(list_3cls)
n_3cls = (len((list_3cls)))
n_elem_group = math.floor((n_test_samples-n_3cls)/n_3cls)
print(n_elem_group)
list_left=[]

for n in range(0, lim):
    if n not in list_3cls:
        list_left.append(n)
        
list_left = np.random.permutation(np.array(list_left))
n_list_left = len(list_left)       
B = np.reshape(list_left[:(n_elem_group*n_3cls)],(-1,n_elem_group))
A = np.reshape(list_3cls,(-1,1))

print(B.shape)
print(A.shape)

groups = np.concatenate((A, B), axis=1)

print (groups)

precision_background_list=[]
precision_cancer_list=[]
precision_normal_list=[]

recall_background_list=[]
recall_cancer_list=[]
recall_normal_list=[]

f1_background_list=[]
f1_cancer_list=[]
f1_normal_list=[]

for i in range(0,n_3cls):
    TP_b=0
    FP_b=0
    FN_b=0
    TN_b=0

    TP_n=0
    FP_n=0
    FN_n=0
    TN_n=0

    TP_c=0
    FP_c=0
    FN_c=0
    TN_c=0
    for j in range(0,n_elem_group+1):
           
        true_positives= np.zeros(shape=(512,512,3))
        false_positives= np.zeros(shape=(512,512,3))
        false_negatives= np.zeros(shape=(512,512,3))
        test_res_sample =  np.array(rgb_to_onehot_3cls(CRF_res[groups[i,j]]))
        
        ground_truth = rgb_to_onehot_3cls(masks[groups[i,j]])
        most_prob = np.zeros(shape = [patch_size, patch_size, n_classes])

        most_prob[:,:,0] = (np.array(test_res_sample[:,:,0] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
        most_prob[:,:,1] = (np.array(test_res_sample[:,:,1] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
        most_prob[:,:,2] = (np.array(test_res_sample[:,:,2] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
        
        for n in range(0,3):
            true_positives[:,:,n] = np.multiply(most_prob[:,:,n], ground_truth[:,:,n])
            false_positives[:,:,n] = (most_prob[:,:,n]-1) == ground_truth[:,:,n]
            false_negatives[:,:,n] = most_prob[:,:,n] == (ground_truth[:,:,n] - 1)

        precision_background = TP_b/(TP_b+FP_b+epsi)
        recall_background = TP_b/(TP_b+FN_b+epsi)
        f1_background = 2 * (precision_background * recall_background/(precision_background + recall_background + epsi))

        TP_b=TP_b+ np.sum(np.sum(np.sum(true_positives[:,:,0])))
        FP_b=FP_b+ np.sum(np.sum(np.sum(false_positives[:,:,0])))
        FN_b=FN_b+ np.sum(np.sum(np.sum(false_negatives[:,:,0])))

        TP_n+=np.sum(np.sum(np.sum(true_positives[:,:,1])))
        FP_n+=np.sum(np.sum(np.sum(false_positives[:,:,1])))
        FN_n+=np.sum(np.sum(np.sum(false_negatives[:,:,1])))

        TP_c+=np.sum(np.sum(np.sum(true_positives[:,:,2])))
        FP_c+=np.sum(np.sum(np.sum(false_positives[:,:,2])))
        FN_c+=np.sum(np.sum(np.sum(false_negatives[:,:,2])))

    if i==n_3cls-1 and j==n_elem_group:
        
        for t in range(n_elem_group*n_3cls ,n_list_left):
            test_res_sample =  np.array(test_res[t,:,:,:])
            ground_truth = rgb_to_onehot_3cls(masks[t])
            most_prob = np.zeros(shape = [patch_size, patch_size, n_classes])
            
            most_prob[:,:,0] = (np.array(test_res_sample[:,:,0] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
            most_prob[:,:,1] = (np.array(test_res_sample[:,:,1] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
            most_prob[:,:,2] = (np.array(test_res_sample[:,:,2] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
        
            for n in range(0,3):
                true_positives[:,:,n] = np.multiply(most_prob[:,:,n], ground_truth[:,:,n])
                false_positives[:,:,n] = (most_prob[:,:,n]-1) == ground_truth[:,:,n]
                false_negatives[:,:,n] = most_prob[:,:,n] == (ground_truth[:,:,n] - 1)

            precision_background = TP_b/(TP_b+FP_b+epsi)
            recall_background = TP_b/(TP_b+FN_b+epsi)
            f1_background = 2 * (precision_background * recall_background/(precision_background + recall_background + epsi))

            TP_b=TP_b+ np.sum(np.sum(np.sum(true_positives[:,:,0])))
            FP_b=FP_b+ np.sum(np.sum(np.sum(false_positives[:,:,0])))
            FN_b=FN_b+ np.sum(np.sum(np.sum(false_negatives[:,:,0])))

            TP_n+=np.sum(np.sum(np.sum(true_positives[:,:,1])))
            FP_n+=np.sum(np.sum(np.sum(false_positives[:,:,1])))
            FN_n+=np.sum(np.sum(np.sum(false_negatives[:,:,1])))

            TP_c+=np.sum(np.sum(np.sum(true_positives[:,:,2])))
            FP_c+=np.sum(np.sum(np.sum(false_positives[:,:,2])))
            FN_c+=np.sum(np.sum(np.sum(false_negatives[:,:,2])))
      
    precision_normal = TP_n/(TP_n+FP_n+epsi)
    recall_normal = TP_n/(TP_n+FN_n+epsi)
    f1_normal = 2 * (precision_normal * recall_normal/(precision_normal + recall_normal + epsi))

    precision_cancer = TP_c/(TP_c+FP_c+epsi)
    recall_cancer = TP_c/(TP_c+FN_c+epsi)
    f1_cancer = 2 * (precision_cancer * recall_cancer/(precision_cancer + recall_cancer + epsi))

    precision_background_list.append(precision_background)
    precision_cancer_list.append(precision_cancer)
    precision_normal_list.append(precision_normal)

    recall_background_list.append(recall_background)
    recall_cancer_list.append(recall_cancer)
    recall_normal_list.append(recall_normal)

    f1_background_list.append(f1_background)
    f1_cancer_list.append(f1_cancer)
    f1_normal_list.append(f1_normal)


    
            

            
            



In [ ]:
print(len(f1_background_list))
print(n_3cls)

In [ ]:
# exp='B'
csv_file = 'Experiments_results_crossFINAL_CRF12_'+exp+'_groups.csv'
if os.path.exists(csv_file) == 0:
    print('The file does not exist, New file is made')
    with open(csv_file, 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)

        filewriter.writerow(['Exp','Source', 'Metric', 'Value'])
        for i in range(0,n_3cls):
            filewriter.writerow([exp,'Background', 'Precision', precision_background_list[i]])
            filewriter.writerow([exp,'Background', 'Recall', recall_background_list[i]])
            filewriter.writerow([exp,'Background', 'F1 score', f1_background_list[i]])
        for i in range(0,n_3cls):
            filewriter.writerow([exp,'Normal', 'Precision', precision_normal_list[i]])
            filewriter.writerow([exp,'Normal', 'Recall', recall_normal_list[i]])
            filewriter.writerow([exp,'Normal', 'F1 score', f1_normal_list[i]])
        for i in range(0,n_3cls):
            filewriter.writerow([exp,'Cancer', 'Precision', precision_cancer_list[i]])
            filewriter.writerow([exp,'Cancer', 'Recall', recall_cancer_list[i]])
            filewriter.writerow([exp,'Cancer', 'F1 score', f1_cancer_list[i]])

In [ ]:
import seaborn as sns

# colors = {'Background': 'white', 'Normal':('#9FD3F9'), 'Cancer':('#E7C0B2')}
colors = {'Background': 'white', 'Normal':('#FF00FF'), 'Cancer':'black'}

df = pd.read_csv(csv_file)

fig = plt.figure()
sns.set(style='ticks')
# col = artist.get_facecolor()
# artist.setedgecolor(col)
ax = plt.subplot(111)
ax = sns.boxplot(x="Metric", y="Value", hue="Source", data=df, palette=colors, fliersize='0.4',showmeans=True)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.12), ncol=3)
plt.xlabel(None)
plt.ylabel('Performance score')
plt.setp(ax.lines, color='gray')
sns.despine(offset=0., trim=True)
plt.savefig('CrossFINAL_CRF12_'+exp+'_groups.png')

plt.show()
plt.clf()
plt.close()



# metrics per image

In [ ]:
# Calculate precision and recall per tissue type
epsi=0.00000001
masks = [cv2.imread(file) for file in sorted(glob.glob(os.path.join(Test_path+'/Masks/Group1/', '*-labels.png')))]


for i in range(0, len(masks)):
    #b,g,r = cv2.split(masks[i])
    masks[i] = (cv2.cvtColor(masks[i], cv2.COLOR_BGR2RGB))

lim=n_test_samples


TP_b=0
FP_b=0
FN_b=0
TN_b=0

TP_n=0
FP_n=0
FN_n=0
TN_n=0

TP_c=0
FP_c=0
FN_c=0
TN_c=0


for n in range(0, lim): #with scalling
    
    true_positives= np.zeros(shape=(512,512,3))
    false_positives= np.zeros(shape=(512,512,3))
    false_negatives= np.zeros(shape=(512,512,3))
    test_res_sample =  np.array(rgb_to_onehot_3cls(CRF_res[n]))
    ground_truth = rgb_to_onehot_3cls(masks[n])
    most_prob = np.zeros(shape = [patch_size, patch_size, n_classes])
    
    most_prob[:,:,0] = (np.array(test_res_sample[:,:,0] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
    most_prob[:,:,1] = (np.array(test_res_sample[:,:,1] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))
    most_prob[:,:,2] = (np.array(test_res_sample[:,:,2] == np.fmax(np.fmax(test_res_sample[:,:,0], test_res_sample[:,:,1]),((test_res_sample[:,:,2]))),dtype = 'float32'))

    for i in range(0,3):
        true_positives[:,:,i] = np.multiply(most_prob[:,:,i], ground_truth[:,:,i])
        false_positives[:,:,i] = (most_prob[:,:,i]-1) == ground_truth[:,:,i]
        false_negatives[:,:,i] = most_prob[:,:,i] == (ground_truth[:,:,i] - 1)
        true_negatives = np.multiply((most_prob - 1),(ground_truth - 1))
        
    TP_b=TP_b+ np.sum(np.sum(np.sum(true_positives[:,:,0])))
    FP_b=FP_b+ np.sum(np.sum(np.sum(false_positives[:,:,0])))
    FN_b=FN_b+ np.sum(np.sum(np.sum(false_negatives[:,:,0])))
    TN_b=TN_b+ np.sum(np.sum(np.sum(true_negatives[:,:,0])))

    TP_n+=np.sum(np.sum(np.sum(true_positives[:,:,1])))
    FP_n+=np.sum(np.sum(np.sum(false_positives[:,:,1])))
    FN_n+=np.sum(np.sum(np.sum(false_negatives[:,:,1])))
    TN_n+=np.sum(np.sum(np.sum(true_negatives[:,:,1])))

    TP_c+=np.sum(np.sum(np.sum(true_positives[:,:,2])))
    FP_c+=np.sum(np.sum(np.sum(false_positives[:,:,2])))
    FN_c+=np.sum(np.sum(np.sum(false_negatives[:,:,2])))
    TN_c+=np.sum(np.sum(np.sum(true_negatives[:,:,2])))

precision_background = TP_b/(TP_b+FP_b+epsi)
recall_background = TP_b/(TP_b+FN_b+epsi)
specificity_background = TN_b/(TN_b+FP_b+epsi)
f1_background = 2 * (precision_background * recall_background/(precision_background + recall_background + epsi))


precision_normal = TP_n/(TP_n+FP_n+epsi)
recall_normal = TP_n/(TP_n+FN_n+epsi)
specificity_normal = TN_n/(TN_n+FP_n+epsi)
FNR_n=FN_n/(FN_n+TP_n+epsi)
f1_normal = 2 * (precision_normal * recall_normal/(precision_normal + recall_normal + epsi))


precision_cancer = TP_c/(TP_c+FP_c+epsi)
recall_cancer = TP_c/(TP_c+FN_c+epsi)
specificity_cancer = TN_c/(TN_c+FP_c+epsi)
f1_cancer = 2 * (precision_cancer * recall_cancer/(precision_cancer + recall_cancer + epsi))




In [ ]:
print((f1_background))
print((f1_normal))


print(recall_cancer)
print(precision_cancer)
print((f1_cancer))


print(specificity_normal)
print(specificity_cancer)
print(specificity_background)


In [ ]:
# np.sum(np.sum(np.sum(false_negatives[:,:,2])))
np.sum(np.sum(np.sum(most_prob[:,:,2])))

In [ ]:
csv_file = 'Experiments_crossFINAL_CRF12_Good_total_mean.csv'
if os.path.exists(csv_file) == 0:
    print('The file does not exist, New file is made')
    with open(csv_file, 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)

        filewriter.writerow(['Exp','Source', 'Metric', 'Value'])
        filewriter.writerow([exp,'Background', 'Precision', precision_background])
        filewriter.writerow([exp,'Background', 'Recall', recall_background])
        filewriter.writerow([exp,'Background', 'F1 score', f1_background])
        filewriter.writerow([exp,'Normal', 'Precision', precision_normal])
        filewriter.writerow([exp,'Normal', 'Recall', recall_normal])
        filewriter.writerow([exp,'Normal', 'F1 score', f1_normal])
        filewriter.writerow([exp,'Cancer', 'Precision', precision_cancer])
        filewriter.writerow([exp,'Cancer', 'Recall', recall_cancer])
        filewriter.writerow([exp,'Cancer', 'F1 score', f1_cancer])
else:
    print('The file exists, New rows are added')
    with open(csv_file, 'a') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        filewriter.writerow([exp,'Background', 'Precision', precision_background])
        filewriter.writerow([exp,'Background', 'Recall', recall_background])
        filewriter.writerow([exp,'Background', 'F1 score', f1_background])
        filewriter.writerow([exp,'Normal', 'Precision', precision_normal])
        filewriter.writerow([exp,'Normal', 'Recall', recall_normal])
        filewriter.writerow([exp,'Normal', 'F1 score', f1_normal])
        filewriter.writerow([exp,'Cancer', 'Precision', precision_cancer])
        filewriter.writerow([exp,'Cancer', 'Recall', recall_cancer])
        filewriter.writerow([exp,'Cancer', 'F1 score', f1_cancer])

In [ ]:
#only after the end
colors = {'Background': 'white', 'Normal':('#FF00FF'), 'Cancer':('black')}


df = pd.read_csv(csv_file)

fig = plt.figure()
sns.set(style='ticks')
ax = plt.subplot(111)
ax = sns.swarmplot(x="Metric", y="Value", hue="Source",palette=colors, data=df, dodge=True, linewidth=0.6)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.13), ncol=3)

plt.xlabel(None)
ax.axis([None, None, -0.05, 1.05])
plt.ylabel('Performance score')
plt.setp(ax.lines, color='gray')
# plt.title('Performance score in five models', y=1.15)
sns.despine(offset=0.5, trim=True)
plt.savefig('CrossFINAL_CRF12_good_all.png')



plt.show()
plt.clf()
plt.close()


